<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>From local Spark MLlib model to cloud with watson-machine-learning-client</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
   </tr>
   <tr style="border: none">
       <th style="border: none"><img src="https://www-03.ibm.com/services/learning/images/journey_watson_wat_machine_learning_600x280.jpg" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps and code to work with **``watson-machine-learning-client``** library. This notebook introduces commands for getting data, model persistance to Watson Machine Learning repository, model deployment and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3 and Apache® Spark 2.0.

You will use publicly available data "GoSales Transactions for Naive Bayes Model" about anonymous outdoor equipment purchase to predict clients' interests in terms of product line like: golf accessories, camping equipement and others.


## Learning goals

The learning goals of this notebook are:
- Load and explore data
- Prepare data for training and evaluation
- Create spark ml pipeline
- Train and evaluate model
- Persist local model in Watson Machine Learning repository using client library
- Deploy model for online scoring using client library
- Score sample records using client library


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Load and explore data](#load)
3.	[Create an Apache® Spark machine learning model](#model)
4.	[Persist created model using api client](#persistance)
5.	[Deploy and score in a Cloud using api client](#scoring)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered). 
-  Make sure that you are using a Spark 2.0 kernel.

<a id="load"></a>
## 2. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform a basic exploration.

Load the data to the Spark DataFrame by using *wget* to upload the data to gpfs and then *read* method. 

**Example**: First, you need to install required packages. You can do it by running the following code. Run it only one time.

In [1]:
import wget
import json
import os

In [2]:
filename = 'GoSales_Tx_NaiveBayes.csv'

if not os.path.isfile(filename):
    link_to_data = 'https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600'
    filename = wget.download(link_to_data)

print(filename)

GoSales_Tx_NaiveBayes.csv


The csv file GoSales_Tx_NaiveBayes.csv is availble on gpfs now. Load the file to Apache® Spark DataFrame using code below.

In [3]:
#spark = SparkSession.builder.getOrCreate()

df_data = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .option('inferSchema', 'true')\
  .load(filename)

Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [4]:
df_data.printSchema()

root
 |-- PRODUCT_LINE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- PROFESSION: string (nullable = true)



As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [5]:
df_data.show()

+--------------------+------+---+--------------+------------+
|        PRODUCT_LINE|GENDER|AGE|MARITAL_STATUS|  PROFESSION|
+--------------------+------+---+--------------+------------+
|Personal Accessories|     M| 27|        Single|Professional|
|Personal Accessories|     F| 39|       Married|       Other|
|Mountaineering Eq...|     F| 39|       Married|       Other|
|Personal Accessories|     F| 56|   Unspecified| Hospitality|
|      Golf Equipment|     M| 45|       Married|     Retired|
|      Golf Equipment|     M| 45|       Married|     Retired|
|   Camping Equipment|     F| 39|       Married|       Other|
|   Camping Equipment|     F| 49|       Married|       Other|
|  Outdoor Protection|     F| 49|       Married|       Other|
|      Golf Equipment|     M| 47|       Married|     Retired|
|      Golf Equipment|     M| 47|       Married|     Retired|
|Mountaineering Eq...|     M| 21|        Single|      Retail|
|Personal Accessories|     F| 66|       Married|       Other|
|   Camp

In [6]:
df_data.count()

60252

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 3. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 3.1: Prepare data

In this subsection you will split your data into: train, test and predict datasets.

In [7]:
splitted_data = df_data.randomSplit([0.8, 0.2], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]

print("Number of training records: " + str(train_data.count()))
print("Number of testing records : " + str(test_data.count()))

Number of training records: 48176
Number of testing records : 12076


As you can see our data has been successfully split into three datasets: 

-  The train data set, which is the largest group, is used for training.
-  The test data set will be used for model evaluation and is used to test the assumptions of the model.

### 3.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.

In [9]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [10]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [11]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, indexed labels back to original labels.

In [12]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [13]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [14]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [15]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)

print("Accuracy = %g" % accuracy)

Accuracy = 0.585045


You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 4. Persist created model with ``watson-machine-learning-client``

In this section you will learn how to store your model in Watson Machine Learning repository by using common python client.

#### First, you must install client libraries from pypi

In [16]:
!pip install watson-machine-learning-client --upgrade

Requirement already up-to-date: watson-machine-learning-client in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages
Requirement already up-to-date: lomond in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tabulate in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: tqdm in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: requests in /gpfs/global_fs01/sym_shared/YPProdSpark/user/sec7-2ac43fd194bc9a-8ef090487f81/.local/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement already up-to-date: urllib3 in

Import installed client by running below code.

In [17]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

**You can check sphinx documentation for watson-machine-learning-client by visiting [documentation webpage](https://wml-api-pyclient.mybluemix.net).**

### 4.1: Work with your instance

Authenticate to Watson Machine Learning service on Bluemix.

**Action**: Put authentication information from your instance of Watson Machine Learning service here.</div>

In [19]:
wml_credentials={
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "*****",
  "username": "*****",
  "password": "*****",
  "instance_id": "*****"
}

**Tip**: Credentials can be found on **Service Credentials** tab of service instance created in Bluemix. If you cannot see **instance_id** field in **Serice Credentials** generate new credentials by pressing **New credential (+)** button. 

#### Create API client by running below code.

In [20]:
client = WatsonMachineLearningAPIClient(wml_credentials)

#### Get instance details.

In [21]:
instance_details = client.service_instance.get_details()

print(json.dumps(instance_details, indent=2))

{
  "metadata": {
    "created_at": "2018-01-30T10:39:51.938Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e",
    "modified_at": "2018-02-02T12:38:53.234Z",
    "guid": "fd1db635-c9f3-4d83-afc7-32efec367e5e"
  },
  "entity": {
    "region": "us-south",
    "account": {
      "id": "8dba8395f5b8dc6fcab49d43161ab9f1",
      "type": "TRIAL",
      "name": "Wojciech Sobala's Account"
    },
    "deployments": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/deployments"
    },
    "source": "Bluemix",
    "status": "Active",
    "plan_id": "0f2a3c2c-456b-40f3-9b19-726d2740b11c",
    "published_models": {
      "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models"
    },
    "organization_guid": "a7b3ef43-5dc2-41fc-8fa7-29b0846e5807",
    "plan": "standard",
    "space_guid": "a6b628f9-3f64-45c0-bf38-739876b63fec",
 

### 4.2: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

In [22]:
model_props = {client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
               client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
               client.repository.ModelMetaNames.NAME: "LOCALLY created Product Line Prediction model"}
published_model = client.repository.store_model(model=model_rf, pipeline=pipeline_rf, meta_props=model_props, training_data=train_data)

### 4.3: Get model details

In [23]:
published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "metadata": {
    "created_at": "2018-02-02T12:43:40.602Z",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/306c00ce-fea6-43cc-8732-36aa095622b3",
    "modified_at": "2018-02-02T12:43:40.660Z",
    "guid": "306c00ce-fea6-43cc-8732-36aa095622b3"
  },
  "entity": {
    "latest_version": {
      "created_at": "2018-02-02T12:43:40.660Z",
      "url": "https://ibm-watson-ml.mybluemix.net/v3/ml_assets/models/306c00ce-fea6-43cc-8732-36aa095622b3/versions/37fd5c86-40fd-40a3-a79b-ed4a16e7d44c",
      "guid": "37fd5c86-40fd-40a3-a79b-ed4a16e7d44c"
    },
    "runtime_environment": "spark-2.0",
    "name": "LOCALLY created Product Line Prediction model",
    "learning_configuration_url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/306c00ce-fea6-43cc-8732-36aa095622b3/learning_configuration",
    "model_type": "mllib-2.0",
    "input_data_schema": {
      "type

#### Get all models

In [24]:
client.repository.list_models()

------------------------------------  ---------------------------------------------  ------------------------  ---------  -----
GUID                                  NAME                                           CREATED                   FRAMEWORK  TYPE
306c00ce-fea6-43cc-8732-36aa095622b3  LOCALLY created Product Line Prediction model  2018-02-02T12:43:40.602Z  mllib-2.0  model
------------------------------------  ---------------------------------------------  ------------------------  ---------  -----


### 4.3: Load model

In this subsection you will learn how to load back saved model from specified instance of Watson Machine Learning.

In [25]:
loaded_model = client.repository.load(published_model_uid)

You can pass test_data to loaded model ``transform()`` method to make sure that model has been loaded correctly.

In [26]:
test_predictions = loaded_model.transform(test_data)

In [27]:
test_predictions.select('probability', 'predictedLabel').show(n=3, truncate=False)

+------------------------------------------------------------------------------------------------------+--------------------+
|probability                                                                                           |predictedLabel      |
+------------------------------------------------------------------------------------------------------+--------------------+
|[0.24720751484299525,0.5408001368079749,0.18043729847153453,0.011111176597498107,0.02044387327999708] |Personal Accessories|
|[0.14405998812488277,0.716936570406598,0.10724091831385933,0.010141281050887725,0.02162124210377206]  |Personal Accessories|
|[0.6552852715561494,0.23794129014749066,0.06795081592072306,0.021435695781928048,0.017386926593708846]|Camping Equipment   |
+------------------------------------------------------------------------------------------------------+--------------------+
only showing top 3 rows



As you can see the loaded model works. You have already learned how save and load the model from Watson Machine Learning repository.

### 4.4: Delete model

You can delete published model from Watson Machine Learning repository using below code. The code is commented out at this stage since the model will be needed later for deployment.

<a id="scoring"></a>
## 5. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the Watson Machine Learning Client. 

### 5.1: Create online deployment

In [28]:
created_deployment = client.deployments.create(published_model_uid, name="Product line prediction")

Now you can print an online scoring endpoint. 

In [29]:
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)

print(scoring_endpoint)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/306c00ce-fea6-43cc-8732-36aa095622b3/deployments/f623c303-43e7-4261-a266-649074436d3e/online


### 5.2: Get deployments

In [30]:
client.deployments.list()

------------------------------------  -----------------------  ------  ------------------------  ---------
GUID                                  NAME                     TYPE    CREATED                   FRAMEWORK
f623c303-43e7-4261-a266-649074436d3e  Product line prediction  online  2018-02-02T12:44:01.960Z  mllib-2.0
------------------------------------  -----------------------  ------  ------------------------  ---------


You can see that your model is deployed, so you can score new data.

### 5.3: Score

You can use below method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [31]:
scoring_payload = {"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"values": [["M",23,"Single","Student"],["M",55,"Single","Executive"]]}

Use ``client.deployments.score()`` method to run scoring.

In [32]:
predictions = client.deployments.score(scoring_endpoint, scoring_payload)

In [33]:
print(json.dumps(predictions, indent=2))

{
  "fields": [
    "GENDER",
    "AGE",
    "MARITAL_STATUS",
    "PROFESSION",
    "PRODUCT_LINE",
    "label",
    "PROFESSION_IX",
    "GENDER_IX",
    "MARITAL_STATUS_IX",
    "features",
    "rawPrediction",
    "probability",
    "prediction",
    "predictedLabel"
  ],
  "values": [
    [
      "M",
      23,
      "Single",
      "Student",
      "Camping Equipment",
      0.0,
      6.0,
      0.0,
      1.0,
      [
        0.0,
        23.0,
        1.0,
        6.0
      ],
      [
        4.5146661875758465,
        7.57017273688273,
        5.662967039590646,
        0.1716508249665755,
        2.0805432109842013
      ],
      [
        0.22573330937879232,
        0.3785086368441365,
        0.2831483519795323,
        0.008582541248328775,
        0.10402716054921006
      ],
      1.0,
      "Personal Accessories"
    ],
    [
      "M",
      55,
      "Single",
      "Executive",
      "Camping Equipment",
      0.0,
      3.0,
      0.0,
      1.0,
      [
        

### 5.4: Delete deployment

Use the following method to delete deployment.

In [34]:
client.deployments.delete(client.deployments.get_uid(created_deployment))

You can check number of depoyments by running ``client.deployment.get()`` method.

In [35]:
client.deployments.list()

----  ----  ----  -------  ---------
GUID  NAME  TYPE  CREATED  FRAMEWORK
----  ----  ----  -------  ---------


### 5.5: Delete model

In [36]:
client.repository.delete(published_model_uid)

You can check number of your models by running below ``client.repository.get_details()`` method.

In [37]:
client.repository.list_models()

----  ----  -------  ---------  ----
GUID  NAME  CREATED  FRAMEWORK  TYPE
----  ----  -------  ---------  ----


<a id="summary"></a>
## 7. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning as well as Watson Machine Learning for model creation and deployment. Check out our [Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, PhD, is a Automation Architect and Data Scientist in IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.